In [168]:
import cupy
from typing import Tuple

In [169]:
def block_stride_view(blocks, view_size: Tuple[int], strides: Tuple[int]=(1,1)):
  batch_size = blocks.shape[0]
  channel = blocks.shape[1]
  domain_height = blocks.shape[2]
  domain_width = blocks.shape[3]

  view_height = view_size[0]
  view_width = view_size[1]
 
  stride_height = strides[0]
  stride_width = strides[1]

  new_width = 1 + int((domain_width - view_width)/stride_width)
  new_height = 1 + int((domain_height - view_height)/stride_height)

  pick_shape = (new_height, new_width, batch_size, channel, view_height, view_width)

  blocks_strides = cupy.array((blocks.strides + blocks.strides)[2:])
  blocks_stride_strides  = cupy.array((blocks.strides[2]*(stride_height-1), blocks.strides[3]*(stride_width-1), 0, 0,0,0))
  # memory_skip = (blocks.strides + blocks.strides)[2:]
  memory_skip = (blocks_strides + blocks_stride_strides).get()

  sub_blocks = cupy.lib.stride_tricks.as_strided(blocks, pick_shape, memory_skip)
  
  return sub_blocks

In [172]:
img = cupy.random.uniform(high=100, size=(20, 3, 16, 16)).astype(int)
sub_blocks = block_stride_view(img, (3,3), (1,1))
print(sub_blocks.shape)

(14, 14, 20, 3, 3, 3)


In [173]:
def reconstruct_blocks(sub_blocks, original_shape: Tuple[int], view_size: Tuple[int], strides: Tuple[int] = (1, 1)):
    # Extract original shape parameters
    batch_size, channels, domain_height, domain_width = original_shape

    # Extract sub_blocks parameters
    new_height = sub_blocks.shape[0]
    new_width = sub_blocks.shape[1]
    view_height = view_size[0]
    view_width = view_size[1]
    
    stride_height = strides[0]
    stride_width = strides[1]

    # Initialize the reconstructed blocks array and a counter array
    blocks = cupy.zeros((batch_size, channels, domain_height, domain_width))
    block_counter = cupy.zeros((batch_size, channels, domain_height, domain_width))

    # Reconstruct the blocks by placing each sub_block at the correct position
    for i in range(new_height):
        for j in range(new_width):
            row_start = i * stride_height
            col_start = j * stride_width
            row_end = row_start + view_height
            col_end = col_start + view_width

            # Add sub_block to the corresponding location in the original blocks
            blocks[:, :, row_start:row_end, col_start:col_end] += sub_blocks[i, j]

    return blocks

In [174]:
blocksx = reconstruct_blocks(sub_blocks, (20,3,16,16), (3,3), (1,1))

In [32]:
import cupy as cp
a = cupy.array([[0,1,2], [1,2,3], [2,3,4], [3,4,5], [5,6,7]])
b = cupy.zeros(shape=(9))


cupy.concatenate(a, axis=0,)

array([0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5, 5, 6, 7])

In [72]:
import cupy as cp

# Define the array
a = cp.array([[[0,1,2],[0,0,0]], 
              [[2,3,4],[0,0,0]],
              [[3,4,5],[0,0,0]],
              [[4,5,6],[0,0,0]],
              [[5,6,7],[0,0,0]]])

# Create a mask that selects the entire first row and only the last element of each subsequent row
mask = cp.array([True, True, True] + [False, False, True] * (a.shape[0] - 1))
mask = cp.reshape(mask, (5,3))
print(mask)
# Flatten the array and apply the mask
result = a[:,:,mask]

# Convert to numpy for easy viewing (if needed)
print(result.get())  # Output on the host

[[ True  True  True]
 [False False  True]
 [False False  True]
 [False False  True]
 [False False  True]]


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [93]:
a[:,:,[[0,1,2],[-1]]]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [126]:
a = cupy.random.normal(size=(10,10))
b = cupy.random.normal(size=(10,12))

c = cupy.einsum('ij,(j+1)k->ik', a, b)

In [124]:
c.shape

(12, 10)

In [136]:
zeta = cupy.random.normal(size=(12,12,20,5,3,3))
psi = cupy.random.normal(size=(5,10,8,8))

sum = cupy.einsum('hwbkat,kcrs', zeta, psi)

In [137]:
sum.shape

(3, 20, 10, 12, 8, 8, 3, 12)